In [58]:
import pandas as pd

In [76]:
df = pd.read_csv("members.csv")

In [77]:
df

,id,name,age,gender,education_level,zip_code,books_borrowed,favorite_genres,date_joined
0,421742,Alan,20,male,high-school-diploma,27509,22,Religion/spirituality;Mystery/thriller/crime,2021-08-26
1,338337,Tessa,27,female,bachelors-associate,27278,0,Science fiction;Graphic novels;History,2021-11-19
2,348835,Flaquita,52,female,high-school-diploma,27252,2,Cookbooks/food writing;Business;History,2021-04-23
3,871610,Eugene,9,male,up-to-high-school,27591,15,Classics/Literature;Biographies/memoirs,2021-09-02
4,793820,Ernesto,64,unspecified,high-school-diploma,27712,20,Biographies/memoirs,2021-02-23
...,...,...,...,...,...,...,...,...,...
54212,512848,Eduardo,58,male,bachelors-associate,27705,18,Biographies/memoirs,2021-02-09
54213,218197,Narciso,53,male,high-school-diploma,27513,41,Current affairs;Biographies/memoirs;Mystery/th...,2021-05-25
54214,403582,Craig,27,male,high-school-diploma,27201,2,True crime;Biographies/memoirs;Cookbooks/food ...,2021-12-01
54215,373414,Abraham,48,male,bachelors-associate,27546,0,Science fiction;Mystery/thriller/crime;Self-help,2021-11-16


In [82]:
df1 = df[['education_level', 'age']].groupby(['education_level']).agg(['mean'])
df1.columns = df1.columns.get_level_values(0)

df1 = df1.sort_values('age')
df1

,age
education_level,
up-to-high-school,17.911909
high-school-diploma,43.677985
bachelors-associate,46.529593
masters-degree,49.689992
doctorate-professional,52.669280


In [62]:
df.shape

(54217, 9)

In [1]:
import os
import requests
from pyspark.sql import SparkSession
from tmlt.analytics.keyset import KeySet
from tmlt.analytics.privacy_budget import PureDPBudget
from tmlt.analytics.query_builder import QueryBuilder
from tmlt.analytics.session import Session

In [64]:
spark = SparkSession.builder.getOrCreate()

members_df = spark.read.csv("members.csv", header=True, inferSchema=True)

session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df,
)

In [65]:
edu_levels = KeySet.from_dict({
    "education_level": [
        "up-to-high-school",
        "high-school-diploma",
        "bachelors-associate",
        "masters-degree",
        "doctorate-professional",
    ] 
})

In [66]:
edu_average_age_query = (
    QueryBuilder("members")
    .groupby(edu_levels)
    .average("age", low=0, high=120)
)


In [67]:
edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(0.1),
)
edu_average_ages.sort("age_average").show(truncate=False)

/home/anshu/Work/tumult_labs/env/lib/python3.7/site-packages/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


+----------------------+------------------+
|education_level       |age_average       |
+----------------------+------------------+
|up-to-high-school     |18.019705102942176|
|high-school-diploma   |43.67578896651409 |
|bachelors-associate   |46.34802988135345 |
|masters-degree        |49.01694452036075 |
|doctorate-professional|52.853067047075605|
+----------------------+------------------+



In [48]:
df1

,age
education_level,
up-to-high-school,17.911909
high-school-diploma,43.677985
bachelors-associate,46.529593
masters-degree,49.689992
doctorate-professional,52.669280


In [68]:
members_df2 = members_df.limit(20000)

In [1]:
session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df2,
)

NameError: name 'Session' is not defined

In [71]:
edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(0.1),
)
edu_average_ages.sort("age_average").show(truncate=False)

+----------------------+------------------+
|education_level       |age_average       |
+----------------------+------------------+
|up-to-high-school     |17.585650723025587|
|high-school-diploma   |43.749186515683974|
|bachelors-associate   |46.45956251254265 |
|doctorate-professional|48.647279549718576|
|masters-degree        |50.98913043478261 |
+----------------------+------------------+



In [72]:
df1

,age
education_level,
up-to-high-school,17.911909
high-school-diploma,43.677985
bachelors-associate,46.529593
masters-degree,49.689992
doctorate-professional,52.669280


In [73]:
members_df3 = members_df.limit(10000)

session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df3,
)

edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(0.1),
)

edu_average_ages.sort("age_average").show(truncate=False)

/home/anshu/Work/tumult_labs/env/lib/python3.7/site-packages/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


+----------------------+------------------+
|education_level       |age_average       |
+----------------------+------------------+
|up-to-high-school     |17.839147998531033|
|high-school-diploma   |43.608460149600205|
|bachelors-associate   |46.895920701486844|
|masters-degree        |49.754772393538914|
|doctorate-professional|55.13333333333333 |
+----------------------+------------------+



In [74]:
df1

,age
education_level,
up-to-high-school,17.911909
high-school-diploma,43.677985
bachelors-associate,46.529593
masters-degree,49.689992
doctorate-professional,52.669280


In [83]:
members_df4 = members_df.limit(5000)

session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df4,
)

edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(0.1),
)

edu_average_ages.sort("age_average").show(truncate=False)

/home/anshu/Work/tumult_labs/env/lib/python3.7/site-packages/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


+----------------------+-----------------+
|education_level       |age_average      |
+----------------------+-----------------+
|up-to-high-school     |20.76109467455621|
|high-school-diploma   |43.95385422128999|
|doctorate-professional|44.87155963302752|
|bachelors-associate   |48.02882305844676|
|masters-degree        |57.42265193370166|
+----------------------+-----------------+



In [84]:
members_df5 = members_df.limit(1000)

session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df5,
)

edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(0.1),
)

edu_average_ages.sort("age_average").show(truncate=False)

+----------------------+------------------+
|education_level       |age_average       |
+----------------------+------------------+
|up-to-high-school     |9.860169491525426 |
|masters-degree        |35.267605633802816|
|high-school-diploma   |45.72119815668203 |
|bachelors-associate   |46.21455938697318 |
|doctorate-professional|72.87878787878788 |
+----------------------+------------------+



In [85]:
df1

,age
education_level,
up-to-high-school,17.911909
high-school-diploma,43.677985
bachelors-associate,46.529593
masters-degree,49.689992
doctorate-professional,52.669280


In [86]:
members_df5 = members_df.limit(500)

session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df5,
)

edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(0.1),
)

edu_average_ages.sort("age_average").show(truncate=False)

/home/anshu/Work/tumult_labs/env/lib/python3.7/site-packages/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


+----------------------+-----------------+
|education_level       |age_average      |
+----------------------+-----------------+
|masters-degree        |-66.95           |
|up-to-high-school     |5.796296296296298|
|doctorate-professional|35.09090909090909|
|bachelors-associate   |38.66666666666667|
|high-school-diploma   |47.62295081967213|
+----------------------+-----------------+



In [87]:
df1

,age
education_level,
up-to-high-school,17.911909
high-school-diploma,43.677985
bachelors-associate,46.529593
masters-degree,49.689992
doctorate-professional,52.669280
